In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from nonlinear_dynamics import simulate
from mld_dynamics import mld, h
from warm_start_hmpc.controller import HybridModelPredictiveController
from warm_start_hmpc.mcais import mcais, solve_dare
from visualizer import vis, animate

In [ ]:
from pympc.dynamics.discrete_time_systems import LinearSystem
from pympc.geometry.polyhedron import Polyhedron

# controller horizon
T = 20

# weight matrices
Q = np.eye(mld.nx) * h
R = np.vstack([1.]+[0.]*(mld.nu - 1)).T * h

# LQR terminal cost
Bu = mld.B[:, :1]
Ru = R[:, :1]
P, K = solve_dare(mld.A, Bu, Q.dot(Q), Ru.dot(Ru))
Q_T = np.linalg.cholesky(P).T
objective = [Q, R, Q_T]

# MCAIS terminal set
A_cl = mld.A + Bu.dot(K)
lhs = mld.F + mld.G[:, :1].dot(K)
rhs = mld.h
terminal_set = mcais(A_cl, lhs, rhs, verbose=True)

# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

## Open-loop solution

In [ ]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

In [ ]:
# solve in open loop
solution, leaves = controller.feedforward(
    x0,
    draw_label='Cart pole with walls'
)[:2]

In [ ]:
x_gurobi, obj_gurobi = controller.feedforward_gurobi(x0)

In [ ]:
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'], h)

## Solve with and without warm start

In [ ]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves = controller.feedforward(
    x1,
    warm_start=warm_start,
#     draw_label='Cart pole with walls warm start'
)

In [ ]:
solution, leaves = controller.feedforward(x1)

## Solve in closed loop

In [ ]:
from visualizer import vis, visualize
vis.jupyter_cell()

In [ ]:
# np.random.seed(1)

# set up simulation
T_sim = 60
x_true = [x0]
x_sim = []
x_predicted = []
u_sim = []
warm_start = None

for t in range(T_sim):
    print('Time step %d.'%t)
    
    # solve miqp
    solution, leaves = controller.feedforward(
        x_true[-1],
        warm_start=warm_start,
#         printing_period=None
    )[:2]
    
    # reorganize solution
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    u0 = np.concatenate((uc0, ub0))
    x_sim_t = simulate(x_true[-1], h, uc0[0])
    x1 = x_sim_t[-1]
#     x1 = solution.variables['x'][1]
    e0 = x1 - mld.A.dot(x_true[-1]) - mld.B.dot(u0)
    print('predicted state:', solution.variables['x'][1])
    print('true state:', x1)
    print('modeling error:', e0)
#     visualize(x1)
    
    # generate warm start
    warm_start = controller.construct_warm_start(leaves, x_true[-1], uc0, ub0, e0)
    
    # retrieve closed-loop trajectory
#     x_sim.append(x1)
    u_sim.append(uc0)
    
    # retrieve closed-loop trajectory
    x_predicted.append(solution.variables['x'][1])
    x_true.append(x1)
    x_sim.extend(x_sim_t)

In [ ]:
indices =  [0,1]
fig = plt.figure()

plt.plot(
    [xt[indices[0]] for xt in x_sim],
    [xt[indices[1]] for xt in x_sim],
    color='c',
    label='Closed-loop trajectory',
    linewidth=3
)

for t in range(T_sim):
    label = 'Next-state prediction'
    if t > 0:
        label=None
    plt.plot(
        [x_true[t][indices[0]], x_predicted[t][indices[0]]],
        [x_true[t][indices[1]], x_predicted[t][indices[1]]],
        c='b',
        label=label
    )
    plt.scatter(
        [x_predicted[t][indices[0]]],
        [x_predicted[t][indices[1]]],
        c='b',
        marker='o',
        s=5,
        zorder=3
    )
plt.xlabel(r'$x_%d$'%(indices[0]+1))
plt.ylabel(r'$x_%d$'%(indices[1]+1))
# ax.set_zlabel(r'Solved QPs')
plt.legend()
plt.grid(True)
# plt.show()
plt.savefig('error_soft.pdf')

## Animation

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(x_sim, h)